In [1]:
"""
Syndrome calculation
"""
from arithmetic_gf import gf_pow
from polynomial_gf import gf_poly_eval

def rs_calc_syndromes(msg, nsym):
    '''Given the received codeword msg and the number of error correcting symbols (nsym), computes the syndromes polynomial.
    Mathematically, it's essentially equivalent to a Fourrier Transform (Chien search being the inverse).
    '''
    # Note the "[0] +" : we add a 0 coefficient for the lowest degree (the constant). This effectively shifts the syndrome, and will shift every computations depending on the syndromes (such as the errors locator polynomial, errors evaluator polynomial, etc. but not the errors positions).
    # This is not necessary, you can adapt subsequent computations to start from 0 instead of skipping the first iteration (ie, the often seen range(1, n-k+1)),
    synd = [0] * nsym
    for i in range(0, nsym):
        synd[i] = gf_poly_eval(msg, gf_pow(2,i))
    return [0] + synd # pad with one 0 for mathematical precision (else we can end up with weird calculations sometimes)

def rs_check(msg, nsym):
    '''Returns true if the message + ecc has no error or false otherwise (may not always catch a wrong decoding or a wrong message, particularly if there are too many errors -- above the Singleton bound --, but it usually does)'''
    return ( max(rs_calc_syndromes(msg, nsym)) == 0 )

In [2]:
def poly_to_str(poly):
    # Theinput is a polynomial in descending order
    return "+".join([f"{x:x}x^{len(poly)-ind-1}" for ind, x in enumerate(poly) if x != 0])

In [11]:
from encoding import rs_encode_msg
msg_in = [0x40, 0xd2, 0x75, 0x47, 0x76, 0x17, 0x32, 0x06,
            0x27, 0x26, 0x96, 0xc6, 0xc6, 0x96, 0x70, 0xec]

msg = rs_encode_msg(msg_in, 10)

poly_to_str(msg)

'40x^25+d2x^24+75x^23+47x^22+76x^21+17x^20+32x^19+6x^18+27x^17+26x^16+96x^15+c6x^14+c6x^13+96x^12+70x^11+ecx^10+bcx^9+2ax^8+90x^7+13x^6+6bx^5+afx^4+efx^3+fdx^2+4bx^1+e0x^0'

In [12]:
synd = rs_calc_syndromes(msg, 10)
print(synd)
print(rs_check(msg,10))

msg[0] = 0  # deliberately damage the message
synd = rs_calc_syndromes(msg, 10) # when corrupted, the syndromes will be non zero
print(synd)
rs_check(msg, 10)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
True
[0, 64, 192, 93, 231, 52, 92, 228, 49, 83, 245]


False